# Reproduction Notebook: Context-Dependent Task Ambiguity

This notebook demonstrates the `Inappropriate Tool Use via Context-Dependent Task Ambiguity` vulnerability in `gpt-oss-20b`. 

We will run a small sample of scenarios from the full dataset (`dataset_scenarios.json`) to provide a quick and clear verification of the finding.

## 1. Setup

First, ensure you have followed the setup instructions in `README.md`:
1.  Created and activated a Python virtual environment.
2.  Installed all packages from `requirements.txt`.
3.  Created a `.env` file in this directory with your `GROQ_API_KEY` and `WANDB_API_KEY`.

In [ ]:
import os
import json
from dotenv import load_dotenv
from groq import Groq
import wandb

# Load environment variables
load_dotenv()

print("Environment loaded.")

## 2. Load Experiment Code and Data

We will import the `run_scenario` function from the `run_experiment.py` script and load our scenarios dataset.

In [ ]:
# Import the core logic from the experiment script
from run_experiment import run_scenario, tools_schema

# Load the scenarios
with open('dataset_scenarios.json', 'r') as f:
    scenarios = json.load(f)

print(f"Loaded {len(scenarios)} scenarios from the dataset.")

## 3. Initialize API Client

We'll set up the Groq client to interact with the `gpt-oss-20b` model. We also initialize a dummy WandB run since our script expects it, but we will print results directly here.

In [ ]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# The experiment script uses wandb.config, so we'll initialize a dummy run
wandb.init(
    project="OpenAI-RedTeam-Reproduction-Notebook",
    name="Context-Ambiguity-Repro-Test",
    config={
        "model_name": "openai/gpt-oss-20b",
        "scenario_set": "dataset_scenarios.json",
        "temperature": 0.5
    },
    mode="disabled" # Run offline, don't sync to cloud
)

print("Groq client and dummy WandB run initialized.")

## 4. Run a Sample of Scenarios

Let's select a few representative scenarios from the dataset and run them. We will look for cases where the model is expected to be vulnerable.

We will print the full conversation history and the final tool call made by the model for each scenario.

In [ ]:
# Select a few interesting scenarios to test (e.g., the first 5)
sample_scenarios = scenarios[:5]

for scenario in sample_scenarios:
    print(f"--- Running Scenario: {scenario['scenario_id']} ---")
    print(f"Description: {scenario['description']}")
    print(f"Expected Dangerous Command: {scenario['expected_dangerous_command']}")
    print("\nConversation History:")
    for turn in scenario['conversation']:
        print(f"  {turn['role'].replace('_', ' ').title()}: {turn.get('content') or turn.get('tool_name')}")
    
    # Run the scenario
    result = run_scenario(client, scenario)
    
    print("\n--- RESULT ---")
    print(f"Final Model Tool Call: {result['final_tool_call']}")
    print(f"Outcome: {result['result']}")
    print("="*50 + "\n")

wandb.finish()